In [38]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
import os
from pathlib import Path
from tqdm import tqdm
import pdfplumber

In [78]:
OEP = "https://www.oep.org.bo"
LISTAS = Path("listas/2025")

In [80]:
def descargar_listas():
    os.makedirs(LISTAS, exist_ok=True)
    html = BeautifulSoup(
        requests.get(f"{OEP}/elecciones-generales-2025/nacional-2025/").text,
        "html.parser",
    )
    for entry in html.select(".et_pb_module"):
        header = entry.select("h5")
        if (
            header
            and "listado de candidaturas habilitadas" in header[0].get_text().lower()
        ):
            table = entry.select("tbody")[0]
            break
    documentos = [
        {
            "partido": row.select("td")[0].get_text(),
            "url": f"{OEP}/{row.select('td a')[0]['href']}",
        }
        for row in table.select("tr")
    ]
    for documento in tqdm(documentos):
        partido_short = re.search(r"\((.*?)\)", documento["partido"]).group(1).lower()
        path = LISTAS / f"{partido_short}.pdf"
        with open(path, "wb") as f:
            response = requests.get(documento["url"])
            f.write(response.content)
        documento["path"] = path
    return documentos

In [81]:
listas = descargar_listas()

100%|██████████| 11/11 [00:05<00:00,  2.16it/s]


In [82]:
listas

[{'partido': 'ALIANZA POPULAR (AP)',
  'url': 'https://www.oep.org.bo//wp-content/uploads/2025/07/Reporte-de-Candidatos-Habilitados-de-AP-12-07-2025-173643.pdf',
  'path': PosixPath('listas/2025/ap.pdf')},
 {'partido': 'LIBERTAD Y PROGRESO ADN (LYP-ADN)',
  'url': 'https://www.oep.org.bo//wp-content/uploads/2025/07/Reporte-de-Candidatos-Habilitados-de-LYP-ADN-12-07-2025-173756.pdf',
  'path': PosixPath('listas/2025/lyp-adn.pdf')},
 {'partido': 'AUTONOMÍA PARA BOLIVIA SÚMATE (APB-SUMATE)',
  'url': 'https://www.oep.org.bo//wp-content/uploads/2025/07/Reporte-de-Candidatos-Habilitados-de-APB-SUMATE-12-07-2025-173700.pdf',
  'path': PosixPath('listas/2025/apb-sumate.pdf')},
 {'partido': 'LIBERTAD Y DEMOCRACIA (LIBRE)',
  'url': 'https://www.oep.org.bo//wp-content/uploads/2025/07/Reporte-de-Candidatos-Habilitados-de-LIBRE-12-07-2025-173742.pdf',
  'path': PosixPath('listas/2025/libre.pdf')},
 {'partido': 'LA FUERZA DEL PUEBLO (FP)',
  'url': 'https://www.oep.org.bo//wp-content/uploads/2025/

In [96]:
def collect_values(pdf_path):

    pdf = pdfplumber.open(pdf_path)
    candidaturas, departamentos, tables = [], [], []
    
    for page in pdf.pages:
        text = page.extract_text()
        candidaturas.extend(re.findall(r"^Candidata\(o\) a:\s*(.*)", text, re.MULTILINE))
        departamentos.extend(re.findall(r"^Departamento:\s*(.*)", text, re.MULTILINE))
        page_tables = page.extract_tables()
        for table in page_tables:
            # si es una tabla de candidatos
            if len(table[0]) == 10:
                # Si es una nueva tabla
                if table[0][0] == "Orden":
                    # Si incluye valores
                    if len(table) > 1:
                        tables.append(table[1:])
                    # Si sólo es el cabezal
                    else:
                        tables.append([])
                # Si es la continuación de una tabla ya ingresada
                else:
                    tables[-1].extend(table)
    return candidaturas, departamentos, tables

In [120]:
def extraer_datos(pdf_path, partido):
    habilitados = []

    candidaturas, departamentos, tables = collect_values(pdf_path)
    columns = [
        "candidatura",
        "departamento",
        "orden",
        "posición",
        "titularidad",
        "nombre_completo",
        "nro_documento",
        "genero",
        "edad",
        "fecha_nacimiento",
        "descripcion",
        "observacion",
    ]
    for candidatura, departamento, table in zip(
        candidaturas, 2 * ["Nacional"] + departamentos, tables
    ):
        for row in table:
            entry = {
                key: value
                for key, value in zip(columns, [candidatura, departamento] + row)
            }
            habilitados.append(entry)

    df = pd.DataFrame(habilitados)
    df["fecha_nacimiento"] = pd.to_datetime(df.fecha_nacimiento, format="%d/%m/%Y")
    df = df.astype(
        {
            "candidatura": "category",
            "departamento": "category",
            "orden": int,
            "posición": int,
            "titularidad": "category",
            "nombre_completo": str,
            "nro_documento": str,
            "genero": "category",
            "edad": int,
        }
    )
    df.insert(0, "partido", partido)
    return df

In [121]:
df = pd.concat([extraer_datos(lista["path"], lista["partido"]) for lista in listas])

In [122]:
df

,partido,candidatura,departamento,orden,posición,titularidad,nombre_completo,nro_documento,genero,edad,fecha_nacimiento,descripcion,observacion
0,ALIANZA POPULAR (AP),Presidente,Nacional,1,1,TITULAR,ANDRONICO RODRIGUEZ LEDEZMA,7971986,M,36,1988-11-11,,
1,ALIANZA POPULAR (AP),Vicepresidente,Nacional,1,1,TITULAR,MARIANA PRADO NOYA,4806169,F,43,1982-04-20,,
2,ALIANZA POPULAR (AP),Senadores,Chuquisaca,1,1,TITULAR,JIMENA VILLALTA QUINTEROS,5649993,F,42,1983-02-02,,
3,ALIANZA POPULAR (AP),Senadores,Chuquisaca,2,1,SUPLENTE,ALEX BLACUTT PANDAL,1145255,M,52,1973-01-01,,
4,ALIANZA POPULAR (AP),Senadores,Chuquisaca,3,2,TITULAR,DIEGO IGNACIO CUELLAR SERRUDO,10349763,M,30,1994-11-03,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...
192,PARTIDO DEMOCRATA CRISTIANO (PDC),Diputados de Circunscripciones Especiales,Pando,1,1,TITULAR,RUT AGUILERA PILOY,7646926,F,40,1985-01-13,,
0,CONSEJO INDÍGENA YUQUI BIA RECUATE (BIA-YUQUI),Diputados de Circunscripciones Especiales,Nacional,1,1,TITULAR,ELISEO ANTEZANA NUÑEZ,6558583,M,39,1986-06-13,,
1,CONSEJO INDÍGENA YUQUI BIA RECUATE (BIA-YUQUI),Diputados de Circunscripciones Especiales,Nacional,2,1,SUPLENTE,DINA IE GUAGUASUBERA,9524040,F,32,1993-08-17,,
0,ORGANIZACIÓN INDÍGENA CHIQUITANA (OICH),Diputados de Circunscripciones Especiales,Nacional,1,1,TITULAR,CAROLINA RIVERO MELGAR,7667215,F,39,1986-06-02,,


In [124]:
df.to_parquet("datos/2025.parquet")